In [1]:
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

from quaternion_layers import *
from BatchNormalization import QuaternionBatchNorm2d 
from InstanceNormalization import QuaternionInstanceNorm2d
from ReLu import Relu 

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, in_planes, planes, norm_fn='group', stride=1):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, padding=1, stride=stride)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)

        num_groups = planes // 8

        if norm_fn == 'group':
            self.norm1 = nn.GroupNorm(num_groups=num_groups, num_channels=planes)
            self.norm2 = nn.GroupNorm(num_groups=num_groups, num_channels=planes)
            if not stride == 1:
                self.norm3 = nn.GroupNorm(num_groups=num_groups, num_channels=planes)

        elif norm_fn == 'batch':
            self.norm1 = nn.BatchNorm2d(planes)
            self.norm2 = nn.BatchNorm2d(planes)
            if not stride == 1:
                self.norm3 = nn.BatchNorm2d(planes)

        elif norm_fn == 'instance':
            self.norm1 = nn.InstanceNorm2d(planes)
            self.norm2 = nn.InstanceNorm2d(planes)
            if not stride == 1:
                self.norm3 = nn.InstanceNorm2d(planes)

        elif norm_fn == 'none':
            self.norm1 = nn.Sequential()
            self.norm2 = nn.Sequential()
            if not stride == 1:
                self.norm3 = nn.Sequential()

        if stride == 1:
            self.downsample = None
        
        else:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride), self.norm3)

    def forward(self, x):
        print(f'Input x size: {x.size()}')  # Print input size
        y = x
        print("shape of x in conv1", y.shape)
        y = self.relu(self.norm1(self.conv1(y)))
        print(f'After conv1 and norm1 size of y : {y.size()}')  # Print size after first convolution and normalization
        
        y = self.relu(self.norm2(self.conv2(y)))
        print(f'After conv2 and norm2 sizeof y : {y.size()}')  # Print size after second convolution and normalization

        if self.downsample is not None:
            x = self.downsample(x)
            print(f'After downsample size of x : {x.size()}')  # Print size after downsampling

        output = self.relu(x + y)
        print(f'Output size: {output.size()}')  # Print output size
        
        return output


In [3]:
# Initialize the ResidualBlock
in_channels = 16  # Example input channels
out_channels = 32  # Example output channels
block = ResidualBlock(in_planes=in_channels, planes=out_channels, norm_fn='group', stride=2)

# Create a dummy input tensor with shape (batch_size, in_channels, height, width)
batch_size = 4
height, width = 64, 64  # Example spatial dimensions
input_tensor = torch.randn(batch_size, in_channels, height, width)


print("Input dimensions:", input_tensor.shape)

output_tensor = block(input_tensor)


Input dimensions: torch.Size([4, 16, 64, 64])
Input x size: torch.Size([4, 16, 64, 64])
shape of x in conv1 torch.Size([4, 16, 64, 64])
After conv1 and norm1 size of y : torch.Size([4, 32, 32, 32])
After conv2 and norm2 sizeof y : torch.Size([4, 32, 32, 32])
After downsample size of x : torch.Size([4, 32, 32, 32])
Output size: torch.Size([4, 32, 32, 32])


In [4]:
class BasicEncoder(nn.Module):
    def __init__(self, output_dim=128, norm_fn='batch', dropout=0.0):
        super(BasicEncoder, self).__init__()
        self.norm_fn = norm_fn

        if self.norm_fn == 'group':
            self.norm1 = nn.GroupNorm(num_groups=8, num_channels=64)
            
        elif self.norm_fn == 'batch':
            self.norm1 = nn.BatchNorm2d(64)

        elif self.norm_fn == 'instance':
            self.norm1 = nn.InstanceNorm2d(64)

        elif self.norm_fn == 'none':
            self.norm1 = nn.Sequential()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.relu1 = nn.ReLU(inplace=True)

        self.in_planes = 64
        self.layer1 = self._make_layer(64,  stride=1)
        self.layer2 = self._make_layer(96, stride=2)
        self.layer3 = self._make_layer(128, stride=2)

        # output convolution
        self.conv2 = nn.Conv2d(128, output_dim, kernel_size=1)

        self.dropout = None
        if dropout > 0:
            self.dropout = nn.Dropout2d(p=dropout)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.InstanceNorm2d, nn.GroupNorm)):
                if m.weight is not None:
                    nn.init.constant_(m.weight, 1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def _make_layer(self, dim, stride=1):
        layer1 = ResidualBlock(self.in_planes, dim, self.norm_fn, stride=stride)
        layer2 = ResidualBlock(dim, dim, self.norm_fn, stride=1)
        layers = (layer1, layer2)
        
        self.in_planes = dim
        return nn.Sequential(*layers)


    def forward(self, x):
        print("input in encoder", x.shape)

        # if input is list, combine batch dimension
        is_list = isinstance(x, tuple) or isinstance(x, list)
        if is_list:
            batch_dim = x[0].shape[0]
            x = torch.cat(x, dim=0)
            
        print("input in encoder going through convolution", x.shape)

        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.conv2(x)

        if self.training and self.dropout is not None:
            x = self.dropout(x)

        if is_list:
            x = torch.split(x, [batch_dim, batch_dim], dim=0)

        return x

In [5]:
# Initialize the BasicEncoder
output_dim = 128  # Example output dimension
encoder = BasicEncoder(output_dim=output_dim, norm_fn='batch', dropout=0.1)

# Create a dummy input tensor with shape (batch_size, channels, height, width)
batch_size = 4
input_tensor = torch.randn(batch_size, 3, 256, 256)  # Example input size

# Print input dimensions
print("Input dimensions:", input_tensor.shape)

# Pass the input tensor through the encoder
output_tensor = encoder(input_tensor)

# Print output dimensions
if isinstance(output_tensor, (tuple, list)):
    for idx, out in enumerate(output_tensor):
        print(f"Output dimensions [{idx}]:", out.shape)
else:
    print("Output dimensions:", output_tensor.shape)

Input dimensions: torch.Size([4, 3, 256, 256])
input in encoder torch.Size([4, 3, 256, 256])
input in encoder going through convolution torch.Size([4, 3, 256, 256])
Input x size: torch.Size([4, 64, 128, 128])
shape of x in conv1 torch.Size([4, 64, 128, 128])
After conv1 and norm1 size of y : torch.Size([4, 64, 128, 128])
After conv2 and norm2 sizeof y : torch.Size([4, 64, 128, 128])
Output size: torch.Size([4, 64, 128, 128])
Input x size: torch.Size([4, 64, 128, 128])
shape of x in conv1 torch.Size([4, 64, 128, 128])
After conv1 and norm1 size of y : torch.Size([4, 64, 128, 128])
After conv2 and norm2 sizeof y : torch.Size([4, 64, 128, 128])
Output size: torch.Size([4, 64, 128, 128])
Input x size: torch.Size([4, 64, 128, 128])
shape of x in conv1 torch.Size([4, 64, 128, 128])
After conv1 and norm1 size of y : torch.Size([4, 96, 64, 64])
After conv2 and norm2 sizeof y : torch.Size([4, 96, 64, 64])
After downsample size of x : torch.Size([4, 96, 64, 64])
Output size: torch.Size([4, 96, 6

In [6]:
class SmallEncoder(nn.Module):
    def __init__(self, output_dim=128, norm_fn='batch', dropout=0.0):
        super(SmallEncoder, self).__init__()
        self.norm_fn = norm_fn

        if self.norm_fn == 'group':
            self.norm1 = nn.GroupNorm(num_groups=8, num_channels=32)
            
        elif self.norm_fn == 'batch':
            self.norm1 = nn.BatchNorm2d(32)

        elif self.norm_fn == 'instance':
            self.norm1 = nn.InstanceNorm2d(32)

        elif self.norm_fn == 'none':
            self.norm1 = nn.Sequential()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=7, stride=2, padding=3)
        self.relu1 = nn.ReLU(inplace=True)

        self.in_planes = 32
        self.layer1 = self._make_layer(32,  stride=1)
        self.layer2 = self._make_layer(64, stride=2)
        self.layer3 = self._make_layer(96, stride=2)

        self.dropout = None
        if dropout > 0:
            self.dropout = nn.Dropout2d(p=dropout)
        
        self.conv2 = nn.Conv2d(96, output_dim, kernel_size=1)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.InstanceNorm2d, nn.GroupNorm)):
                if m.weight is not None:
                    nn.init.constant_(m.weight, 1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def _make_layer(self, dim, stride=1):
        layer1 = BottleneckBlock(self.in_planes, dim, self.norm_fn, stride=stride)
        layer2 = BottleneckBlock(dim, dim, self.norm_fn, stride=1)
        layers = (layer1, layer2)
    
        self.in_planes = dim
        return nn.Sequential(*layers)


    def forward(self, x):

        # if input is list, combine batch dimension
        is_list = isinstance(x, tuple) or isinstance(x, list)
        if is_list:
            batch_dim = x[0].shape[0]
            x = torch.cat(x, dim=0)

        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.conv2(x)

        if self.training and self.dropout is not None:
            x = self.dropout(x)

        if is_list:
            x = torch.split(x, [batch_dim, batch_dim], dim=0)

        return x

### MODIFIED ENCODER 


In [7]:
class ResidualBlock1(nn.Module):
    def __init__(self, in_planes, planes, norm_fn='group', stride=1):
        super(ResidualBlock1, self).__init__()
  
        self.conv1 = QuaternionConv(in_planes, planes, kernel_size=3, padding=1, stride=stride)
        self.conv2 = QuaternionConv(planes, planes, kernel_size=3, padding=1 , stride =1)
        self.relu = Relu()

        num_groups = planes // 8

        if norm_fn == 'group':
            self.norm1 = nn.GroupNorm(num_groups=num_groups, num_channels=planes)
            self.norm2 = nn.GroupNorm(num_groups=num_groups, num_channels=planes)
            if not stride == 1:
                self.norm3 = nn.GroupNorm(num_groups=num_groups, num_channels=planes)
        
        elif norm_fn == 'batch':
            self.norm1 = QuaternionBatchNorm2d(planes)
            self.norm2 = QuaternionBatchNorm2d(planes)
            if not stride == 1:
                self.norm3 = QuaternionBatchNorm2d(planes)
        
        elif norm_fn == 'instance':
            self.norm1 = QuaternionInstanceNorm2d(planes)
            self.norm2 = QuaternionInstanceNorm2d(planes)
            if not stride == 1:
                self.norm3 = QuaternionInstanceNorm2d(planes)

        elif norm_fn == 'none':
            self.norm1 = nn.Sequential()
            self.norm2 = nn.Sequential()
            if not stride == 1:
                self.norm3 = nn.Sequential()

        if stride == 1:
            self.downsample = None
        
        else:    
            self.downsample = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride), self.norm3)


    def forward(self, x):
        print(f'Input x size: {x.size()}')  # Print input size
        y = x
        print("shape of x in conv1", y.shape)
        y = self.relu(self.norm1(self.conv1(y)))
        print(f'After conv1 and norm1 size: {y.size()}')  # Print size after first convolution and normalization
        y = self.relu(self.norm2(self.conv2(y)))
        print(f'After conv2 and norm2 size: {y.size()}')  # Print size after second convolution and normalization

        if self.downsample is not None:
            x = self.downsample(x)
            print(f'After downsample size: {x.size()}')  # Print size after downsampling

        return self.relu(x + y)


In [8]:
# Initialize the ResidualBlock
in_channels = 16  # Example input channels
out_channels = 32  # Example output channels
block = ResidualBlock1(in_planes=in_channels, planes=out_channels, norm_fn='batch', stride=2)

# Create a dummy input tensor with shape (batch_size, in_channels, height, width)
batch_size = 4
height, width = 64, 64  # Example spatial dimensions
input_tensor = torch.randn(batch_size, in_channels, height, width)

# Print input dimensions
print("Input dimensions:", input_tensor.shape)

# Pass the input tensor through the block
output_tensor = block(input_tensor)

# Print output dimensions
print("Output dimensions:", output_tensor.shape)

Input dimensions: torch.Size([4, 16, 64, 64])
Input x size: torch.Size([4, 16, 64, 64])
shape of x in conv1 torch.Size([4, 16, 64, 64])
After conv1 and norm1 size: torch.Size([4, 32, 32, 32])
After conv2 and norm2 size: torch.Size([4, 32, 32, 32])
After downsample size: torch.Size([4, 32, 32, 32])
Output dimensions: torch.Size([4, 32, 32, 32])


In [15]:
class BasicEncoder(nn.Module):
    def __init__(self, output_dim=128, norm_fn='batch', dropout=0.0):
        super(BasicEncoder, self).__init__()
        self.norm_fn = norm_fn

        if self.norm_fn == 'group':
            self.norm1 = nn.GroupNorm(num_groups=8, num_channels=64)
            
        elif self.norm_fn == 'batch':
            self.norm1 = QuaternionBatchNorm2d(64)

        elif self.norm_fn == 'instance':
            self.norm1 = QuaternionInstanceNorm2d(64)

        elif self.norm_fn == 'none':
            self.norm1 = nn.Sequential()

        self.conv1 = QuaternionConv(3, 64, kernel_size=7, stride=2, padding=3)
        self.relu1 = Relu()

        self.in_planes = 64
        self.layer1 = self._make_layer(64,  stride=1)
        self.layer2 = self._make_layer(96, stride=2)
        self.layer3 = self._make_layer(128, stride=2)

        # output convolution
        self.conv2 = QuaternionConv(128, output_dim, kernel_size=1)

        self.dropout = None
        if dropout > 0:
            self.dropout = nn.Dropout2d(p=dropout)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.InstanceNorm2d, nn.GroupNorm)):
                if m.weight is not None:
                    nn.init.constant_(m.weight, 1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def _make_layer(self, dim, stride=1):
        layer1 = ResidualBlock(self.in_planes, dim, self.norm_fn, stride=stride)
        layer2 = ResidualBlock(dim, dim, self.norm_fn, stride=1)
        layers = (layer1, layer2)
        
        self.in_planes = dim
        return nn.Sequential(*layers)


    def forward(self, x):
        print("this is the input in encoder", x.shape)

        # if input is list, combine batch dimension
        is_list = isinstance(x, tuple) or isinstance(x, list)
        if is_list:
            batch_dim = x[0].shape[0]
            x = torch.cat(x, dim=0)
            
        
        print("Input shape:", x.shape)

        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.conv2(x)

        if self.training and self.dropout is not None:
            x = self.dropout(x)

        if is_list:
            x = torch.split(x, [batch_dim, batch_dim], dim=0)

        return x

In [16]:
# Initialize the BasicEncoder
output_dim = 128  # Example output dimension
encoder = BasicEncoder(output_dim=output_dim, norm_fn='batch', dropout=0.1)

# Create a dummy input tensor with shape (batch_size, channels, height, width)
batch_size = 4
input_tensor = torch.randn(batch_size, 3, 256, 256)  # Example input size

# Print input dimensions
print("Input dimensions:", input_tensor.shape)

# Pass the input tensor through the encoder
output_tensor = encoder(input_tensor)

# Print output dimensions
if isinstance(output_tensor, (tuple, list)):
    for idx, out in enumerate(output_tensor):
        print(f"Output dimensions [{idx}]:", out.shape)
else:
    print("Output dimensions:", output_tensor.shape)

Input dimensions: torch.Size([4, 3, 256, 256])
this is the input in encoder torch.Size([4, 3, 256, 256])
Input shape: torch.Size([4, 3, 256, 256])


RuntimeError: Given groups=1, weight of size [64, 0, 7, 7], expected input[4, 3, 256, 256] to have 0 channels, but got 3 channels instead

In [17]:
class BottleneckBlock(nn.Module):
    def __init__(self, in_planes, planes, norm_fn='group', stride=1):
        super(BottleneckBlock, self).__init__()
  
        self.conv1 = QuaternionConv(in_planes, planes//4, kernel_size=1, padding=0)
        self.conv2 = QuaternionConv(planes//4, planes//4, kernel_size=3, padding=1, stride=stride)
        self.conv3 = QuaternionConv(planes//4, planes, kernel_size=1, padding=0)
        self.relu = Relu()

        num_groups = planes // 8

        if norm_fn == 'group':
            self.norm1 = nn.GroupNorm(num_groups=num_groups, num_channels=planes//4)
            self.norm2 = nn.GroupNorm(num_groups=num_groups, num_channels=planes//4)
            self.norm3 = nn.GroupNorm(num_groups=num_groups, num_channels=planes)
            if not stride == 1:
                self.norm4 = nn.GroupNorm(num_groups=num_groups, num_channels=planes)
        
        elif norm_fn == 'batch':
            self.norm1 = QuaternionBatchNorm2d(planes//4)
            self.norm2 = QuaternionBatchNorm2d(planes//4)
            self.norm3 = QuaternionBatchNorm2d(planes)
            if not stride == 1:
                self.norm4 = QuaternionBatchNorm2d(planes)
        
        elif norm_fn == 'instance':
            self.norm1 = QuaternionInstanceNorm2d(planes//4)
            self.norm2 = QuaternionInstanceNorm2d(planes//4)
            self.norm3 = QuaternionInstanceNorm2d(planes)
            if not stride == 1:
                self.norm4 = QuaternionInstanceNorm2d(planes)

        elif norm_fn == 'none':
            self.norm1 = nn.Sequential()
            self.norm2 = nn.Sequential()
            self.norm3 = nn.Sequential()
            if not stride == 1:
                self.norm4 = nn.Sequential()

        if stride == 1:
            self.downsample = None
        
        else:    
            self.downsample = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride), self.norm4)


    def forward(self, x):
        y = x
        y = self.relu(self.norm1(self.conv1(y)))
        y = self.relu(self.norm2(self.conv2(y)))
        y = self.relu(self.norm3(self.conv3(y)))

        if self.downsample is not None:
            x = self.downsample(x)

        return self.relu(x+y)

In [18]:
class SmallEncoder(nn.Module):
    def __init__(self, output_dim=128, norm_fn='batch', dropout=0.0):
        super(SmallEncoder, self).__init__()
        self.norm_fn = norm_fn

        if self.norm_fn == 'group':
            self.norm1 = nn.GroupNorm(num_groups=8, num_channels=32)
            
        elif self.norm_fn == 'batch':
            self.norm1 = QuaternionBatchNorm2d(32)

        elif self.norm_fn == 'instance':
            self.norm1 = QuaternionInstanceNorm2d(32)

        elif self.norm_fn == 'none':
            self.norm1 = nn.Sequential()

        self.conv1 = QuaternionConv(3, 32, kernel_size=7, stride=2, padding=3)
        self.relu1 = Relu()

        self.in_planes = 32
        self.layer1 = self._make_layer(32,  stride=1)
        self.layer2 = self._make_layer(64, stride=2)
        self.layer3 = self._make_layer(96, stride=2)

        self.dropout = None
        if dropout > 0:
            self.dropout = nn.Dropout2d(p=dropout)
        
        self.conv2 = QuaternionConv(96, output_dim, kernel_size=1)

        for m in self.modules():
            if isinstance(m, QuaternionConv):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (QuaternionBatchNorm2d, QuaternionInstanceNorm2d, nn.GroupNorm)):
                if m.weight is not None:
                    nn.init.constant_(m.weight, 1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def _make_layer(self, dim, stride=1):
        layer1 = BottleneckBlock(self.in_planes, dim, self.norm_fn, stride=stride)
        layer2 = BottleneckBlock(dim, dim, self.norm_fn, stride=1)
        layers = (layer1, layer2)
    
        self.in_planes = dim
        return nn.Sequential(*layers)


    def forward(self, x):

        # if input is list, combine batch dimension
        is_list = isinstance(x, tuple) or isinstance(x, list)
        if is_list:
            batch_dim = x[0].shape[0]
            x = torch.cat(x, dim=0)

        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.conv2(x)

        if self.training and self.dropout is not None:
            x = self.dropout(x)

        if is_list:
            x = torch.split(x, [batch_dim, batch_dim], dim=0)

        return x

In [19]:
# Initialize the BasicEncoder
output_dim = 128  # Example output dimension
encoder = SmallEncoder(output_dim=output_dim, norm_fn='batch', dropout=0.1)

# Create a dummy input tensor with shape (batch_size, channels, height, width)
batch_size = 4
input_tensor = torch.randn(batch_size, 3, 256, 256)  # Example input size

# Print input dimensions
print("Input dimensions:", input_tensor.shape)

# Pass the input tensor through the encoder
output_tensor = encoder(input_tensor)

# Print output dimensions
if isinstance(output_tensor, (tuple, list)):
    for idx, out in enumerate(output_tensor):
        print(f"Output dimensions [{idx}]:", out.shape)
else:
    print("Output dimensions:", output_tensor.shape)

AttributeError: 'QuaternionBatchNorm2d' object has no attribute 'weight'